In [1]:
%pip install nltk pandas transformers
%pip install SpeechRecognition pydub
%pip install -U jupyter ipywidgets
%pip install --upgrade setuptools
%pip install torch torchvision torchaudio
%pip install tensorflow
%pip install tensorflow --upgrade



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Import required libraries
import speech_recognition as sr
import nltk
import os
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from pydub import AudioSegment
from transformers import pipeline
from concurrent.futures import ThreadPoolExecutor, as_completed

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
No module named 'distutils'

In [4]:
# Ensure NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/hoobot/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hoobot/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Speech to Text

In [5]:
# Convert MP3 to WAV for easier processing by SpeechRecognition
def convert_audio(input_file):
    sound = AudioSegment.from_mp3(input_file)
    output_file = "converted_file.wav"
    sound.export(output_file, format="wav")
    return output_file

In [6]:
 # Transcribe audio to text
def transcribe_audio(file_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(file_path) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
            return text
        except sr.UnknownValueError:
            return "Google Speech Recognition could not understand audio"
        except sr.RequestError as e:
            return "Could not request results from Google Speech Recognition service; {0}".format(e)



In [7]:
# Split video to 30 second chunks for faster processing and less chance of error
def convert_and_chunk_audio(input_file, output_folder="audio_chunks", chunk_length_ms=30000):
    # Create the output folder if it doesn't already exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    sound = AudioSegment.from_mp3(input_file)
    chunks = []

    for i in range(0, len(sound), chunk_length_ms):
        chunk = sound[i:i+chunk_length_ms]
        chunk_name = f"chunk{i//chunk_length_ms}.wav"
        chunk_path = os.path.join(output_folder, chunk_name)  # Save chunks to the specified folder
        chunk.export(chunk_path, format="wav", parameters=["-ar", "16000"])
        chunks.append(chunk_path)

    return chunks

In [8]:
def parallel_transcribe_audio(chunks):
    transcriptions = []
    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_chunk = {executor.submit(transcribe_audio, chunk): chunk for chunk in chunks}
        for future in as_completed(future_to_chunk):
            chunk = future_to_chunk[future]
            try:
                transcription = future.result()
                transcriptions.append(transcription)
            except Exception as exc:
                print(f'Chunk {chunk} generated an exception: {exc}')
    return " ".join(transcriptions)

In [9]:
def process_audio_file(input_file):
    chunks = convert_and_chunk_audio(input_file)
    full_transcription = parallel_transcribe_audio(chunks)
    print("Full Transcription:", full_transcription)

    with open("transcribed_text.txt", "w") as text_file:
        text_file.write(full_transcription)

## Driver Code

In [10]:
# Replace 'Interview_Audio.mp3' with the exact audio file
# process_audio_file("Exit Interview Questions from We.mp3")
process_audio_file("Why I left my job at Google (as a software engineer).mp3")

Full Transcription: what's all the food was free and it was fantastic and when I first got in I couldn't understand why anyone would ever want to leave the place like I had worked so hard to get into Google and it was for me a dream come true I have been applying to try to get in for many years and I had felt the interview process multiple times actually like three times or something and on my last attempt I remember the interviewers even telling me why you must really want to get into Google because our records show that you just keep and you know it was it was great not before I got into Google I would be one of those people who would just go like Costco and just get as many samples as I could and that would be my lunch I would love food samples and I might go Trader Joe's and grab their free coffee and I might go to chocolate shops and just get free samples everywhere and you know I was just hungry and starving basically everywhere I went and after Google that kind of changed me a l

## Sentiment Analysis

In [11]:
# Sentiment Analysis Function
def analyze_sentiment(text):
    # Load the sentiment analysis pipeline
    classifier = pipeline('sentiment-analysis')
    # Analyze the sentiment
    results = classifier(text)
    return results

In [12]:
# Analyze the Sentiment of Transcribed Text
def analyze_transcribed_file(file_path):
    # Assuming you have already run the process_audio_file function and have the transcribed text saved
    with open(file_path, "r") as file:
        transcribed_text = file.read()
        
    # Break the text into smaller chunks if necessary (depending on the size and the limitations of the model)
    # For simplicity, we'll assume the text is short enough to be processed in one go
    sentiment_results = analyze_sentiment(transcribed_text)
    
    # Print the results
    for result in sentiment_results:
        print(f"Sentiment: {result['label']}, Confidence: {result['score']:.2f}")

In [16]:
# Example: Analyze the sentiment of the previously transcribed audio file
transcribed_text_file = "transcribed_text.txt"  # Ensure this is the correct path to your transcribed text file
analyze_transcribed_file(transcribed_text_file)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.